<a href="https://colab.research.google.com/github/Akshat977/Applied-Data-Science-Capstone-Coursera-/blob/main/week%203%20part%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

In [2]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

In [11]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade html5lib

     |████████████████████████████████| 122kB 8.0MB/s 
ERROR: html-table-parser 0.1.0 has requirement beautifulsoup4==4.4.1, but you'll have beautifulsoup4 4.9.3 which is incompatible.
  Found existing installation: beautifulsoup4 4.4.1
    Uninstalling beautifulsoup4-4.4.1:
      Successfully uninstalled beautifulsoup4-4.4.1
     |████████████████████████████████| 112kB 8.1MB/s 
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response.status_code)


200


In [65]:
soup = BeautifulSoup(response.text,"html.parser")

In [64]:
table = soup.findAll('table',{"class":"sortable"})[0]
values =[]
#There are 12 rows out of which we  need only rows from 1 to 11.
tr = table.findAll(['tr'])


In [8]:
csvFile = open("toronto.csv",'wt',newline='',encoding='utf-8')
writer = csv.writer(csvFile)  
try:   
        for cell in tr:
            th = cell.find_all('th')
            th_data = [col.text.strip('\n') for col in th]
            td = cell.find_all('td')
            row = [i.text.replace('\n','') for i in td]
            writer.writerow(th_data+row)      
        
finally:   
    csvFile.close()

In [11]:
df=pd.read_csv('toronto.csv')
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


**CLEANING THE DATAFRAME:**
   

*    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
*   More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


*   If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [21]:
for index,row in df.iterrows():
  if row['Borough']=='Not assigned':
    df=df.drop(index)
  if (row['Borough']!='Not assigned' and row['Neighbourhood']=='Not assigned'):
    row['Neighbourhood']=row['Borough'] 
df.reset_index(inplace=True,drop=True)
df.head(10)     

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [69]:
df.sort_values('Postal Code',ascending=True,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**ADDING LATITUDE AND LONGITUDE TO EACH POSTAL CODE:**

In [62]:
address='M5G, Toronto, Ontario'
location=None
while location is None:
  geolocator=Nominatim(user_agent='foursquare_agent')
  location=geolocator.geocode(address)

KeyboardInterrupt: ignored

**SINCE NEITHER PYTHON GEOCODE NOR geocode.google seem to be orking the latitude and longitude data was taken from a Canadian csv file containing the data:**
http://cocl.us/Geospatial_data/

In [70]:
dff=pd.read_csv('http://cocl.us/Geospatial_data/')
dff.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


***merging the datframes on 'postal code'***

In [71]:
df=pd.merge(df,dff,on='Postal Code')
df.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
